In [1]:
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process import GaussianProcessRegressor

In [2]:
def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

In [3]:
kernel = 1.0 * RBF(1.0)

In [4]:
diabetes = load_diabetes()
X,y = diabetes.data,diabetes.target
X_train,X_test,Y_train,y_test = train_test_split(X,y,test_size=0.2)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [5]:
# Kernel function
def gaussian_kernel(a,b,sigma=1):
    ''' GP squared exponential kernel/Gaussian Kernel Function
    '''
    sqdist = np.sum(a**2,1).reshape(-1,1) + np.sum(b**2,1) - 2*np.dot(a,b.T)
    return np.exp(-.5*(1/sigma)*sqdist)

In [6]:
class GausianProcessRegression():
    ''' Gausian Process for linear regression without noise
    '''
    def __init__(self, kernel_ = gaussian_kernel):
        self.K = kernel_
        self.is_fitted = False 
        
    def fit(self,X,Y):
        '''Train model'''
        N = X.shape[0]
        # covariance matrix
        self.X = X
        self.y = Y
        self.is_fitted = True
        
    def predict(self,X2):
        """
        Calculate the posterior mean and covariance matrix for y2
        based on the corresponding input X2, the observations (y1, X1), 
        and the prior kernel function and assuming mean prior µ1 = 0
        """
        if self.is_fitted == False:
            print("Please fit before predict.")
            return
        
        # Kernel of the observations
        Σ11 = self.K(self.X, self.X)

        # Kernel of observations vs to-predict
        Σ12 = self.K(self.X, X2)

        # Compute posterior mean
        μ2 = Σ12.T.dot(np.linalg.inv(Σ11)) @ self.y

        # Compute the posterior covariance
        Σ22 = self.K(X2, X2)
        Σ2 = Σ22 - (Σ12.T.dot(np.linalg.inv(Σ11)) @ Σ12)
        
        return µ2, Σ2
     

In [7]:
# Train model 
GP = GausianProcessRegression()
GP.fit(X_train,Y_train)
# Test model 
mu,sigma = GP.predict(X_test)

In [8]:
# Train model sklearn
sk_model = GaussianProcessRegressor(kernel=kernel,random_state=0)
sk_model.fit(X_train, Y_train)
# Test model sklearn
sk_mu_pred,sk_sigma = sk_model.predict(X_test,return_cov=True)

In [9]:
mu_error = rel_error(mu, sk_mu_pred)
print("prediction error: ", mu_error)

prediction error:  2.4544850856987885e-10


In [10]:
sigma_error = rel_error(sigma, sk_sigma)
print("prediction error: ", sigma_error)

prediction error:  2.462850849631284e-07
